In [ ]:
import json
import requests
from src import utils
import time
import os

In [ ]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [ ]:
api = utils.read_json_file(api_path)

In [ ]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_preprocess.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [ ]:
output_file_name = 'zero-shot-public_mwp_data_v2_preprocess'

In [ ]:
# os.makedirs(output_path, exist_ok=True)
# output_file_name = 'result-' + input_file.split('/')[-1]
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

In [ ]:
jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
data = utils.read_jsonlines(input_file)

In [ ]:
len(data)

In [ ]:
# api

In [ ]:
# url = api['url']

In [ ]:
# headers = api['headers']

In [ ]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [ ]:
# headers

In [ ]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [ ]:
model = model_api(api['url'], api['headers'])

In [ ]:
BODY_TEMP = api['body']

In [ ]:
# body = json.dumps(body_temp)

In [ ]:
# print(body)

In [ ]:
# response = api.generate(body)

In [ ]:
# context = response.json()['data'][0]['result'][0]

In [ ]:
# print(context)

In [ ]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [ ]:
def get_mwp_zs(que):
    prompt = f'질문: {que} 정답 : '
    return prompt

In [ ]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [ ]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [ ]:
cnt = 0
split_cnt = 0
for ins in data:

    kor_que = ins['question']
    
    
    prompt = get_mwp_zs(kor_que)


    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    
    ins['zs_infer'] = res_context
    jsonl_writer.write_json_line(ins)
    
